# Test of provenance

## Objective
- The goal of this test is to have an idea of the existing tools to implement the provenance in a program of CTA using the ctapipe library

## Context
- This test is done with the muon_reconstruction.py program 
- This program uses Karl ctapipe and provenance modules/libraries (v. 0.5.2.post756+git93dae3f - feature/improve-tool branch on https://github.com/kosack/ctapipe)
- The Provenance database is defined in memory and accessed with the sqlalchemy library

## Structure of the notebook
- Definition the Provenance database structure
- Definition of the muon_reconstruction tool
- Addition of the muon_reconstruction (named ctapipe_display_muons) activity description in the provenance database
- Execution of the muon_reconstruction program
- Addition of the provenance information of the job in the database
- Query the provenance database and store the result in a file
- Vizualisation of the provenance


## Definition of the Provenance database structure

#### Provenance Data Model PR2 2019-07-19
<img src="2019-07-19_PR2_PROV_Fig8.png">

#### Remarks
- Activity
    Activity.activityDescription = concat(activity_name, '_', ctapipe_version)
    Dates are curreuntly stored as strings
- Entity
    Entity.id = hash(file)
    Inheritance is implemented as joined table inheritance (dependant tables) => addition of the classType attribute in the Entity and EntityDescription classes
- Relations
    Used.id, WasGeneratedBy.id, WasAttributedTo.id, WasAssociatedWith.id are interger and autoincremented
- A lot of empty fields and problem to associate Entity with EntityDescription


In [2]:
from provenanceDB import *

## muon_reconstruction definition

In [2]:
"""
Example to load raw data (hessio format), calibrate and reconstruct muon
ring parameters, and write the muon ring and intensity parameters to an output
table.

The resulting output can be read e.g. using `pandas.read_hdf(filename,
'muons/LSTCam')`
"""

import warnings
from collections import defaultdict

from tqdm import tqdm

from ctapipe.calib import CameraCalibrator
from ctapipe.core import Provenance
from ctapipe.core import Tool, ToolConfigurationError
from ctapipe.core import traits as t
from ctapipe.image.muon.muon_diagnostic_plots import plot_muon_event
from ctapipe.image.muon.muon_reco_functions import analyze_muon_event
from ctapipe.io import EventSource, event_source
from ctapipe.io import HDF5TableWriter

warnings.filterwarnings("ignore")  # Supresses iminuit warnings


def _exclude_some_columns(subarray, writer):
    """ a hack to exclude some columns of all output tables here we exclude
    the prediction and mask quantities, since they are arrays and thus not
    readable by pandas.  Also, prediction currently is a variable-length
    quantity (need to change it to be fixed-length), so it cannot be written
    to a fixed-length table.
    """
    all_camids = {str(x.camera) for x in subarray.tel.values()}
    for cam in all_camids:
        writer.exclude(cam, 'prediction')
        writer.exclude(cam, 'mask')

class MuonDisplayerTool(Tool):
    name = 'ctapipe-reconstruct-muons'
    description = t.Unicode(__doc__)

    events = t.Unicode("",
                       help="input event data file").tag(config=True)

    outfile = t.Unicode("muons.hdf5", help='HDF5 output file name').tag(
        config=True)

    display = t.Bool(
        help='display the camera events', default=False
    ).tag(config=True)

    classes = t.List([
        CameraCalibrator, EventSource
    ])

    aliases = t.Dict({
        'input': 'MuonDisplayerTool.events',
        'outfile': 'MuonDisplayerTool.outfile',
        'display': 'MuonDisplayerTool.display',
        'max_events': 'EventSource.max_events',
        'allowed_tels': 'EventSource.allowed_tels',
    })

    def setup(self):
        if self.events == '':
            raise ToolConfigurationError("please specify --input <events file>")
        self.log.debug("input: %s", self.events)
        self.source = event_source(self.events)
        self.calib = CameraCalibrator(parent=self)
        self.writer = HDF5TableWriter(self.outfile, "muons")

    def start(self):

        numev = 0
        self.num_muons_found = defaultdict(int)

        for event in tqdm(self.source, desc='detecting muons'):

            self.calib(event)
            muon_evt = analyze_muon_event(event)

            if numev == 0:
                _exclude_some_columns(event.inst.subarray, self.writer)

            numev += 1

            if not muon_evt['MuonIntensityParams']:
                # No telescopes  contained a good muon
                continue
            else:
                if self.display:
                    plot_muon_event(event, muon_evt)

                for tel_id in muon_evt['TelIds']:
                    idx = muon_evt['TelIds'].index(tel_id)
                    intens_params = muon_evt['MuonIntensityParams'][idx]

                    if intens_params is not None:
                        ring_params = muon_evt['MuonRingParams'][idx]
                        cam_id = str(event.inst.subarray.tel[tel_id].camera)
                        self.num_muons_found[cam_id] += 1
                        self.log.debug("INTENSITY: %s", intens_params)
                        self.log.debug("RING: %s", ring_params)
                        self.writer.write(table_name=cam_id,
                                          containers=[intens_params,
                                                      ring_params])

                self.log.info(
                    "Event Number: %d, found %s muons",
                    numev, dict(self.num_muons_found)
                )

    def finish(self):
        Provenance().add_output_file(self.outfile,
                                     role='dl1.tel.evt.muon')
        self.writer.close()






## Descriptions added in the Provenance database

In [3]:
# Define the session to talk to the database
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

# Create an instance of ActivityDescription 
actDesc1 = ActivityDescription(id='ctapipe_display_muons_0.6.1',name='ctapipe_display_muons',\
                               type='reconstruction',subtype='',version='0.6.1', doculink='')

# Create the description of input entities
dataDesc1 = DatasetDescription(id='proton_events', name='protons', description='proton file', classType='datasetDescription')
usedDesc1 = UsageDescription(id='ctapipe_display_muons_0.6.1_proton_events',activityDescription=actDesc1, entityDescription=dataDesc1, role="dl0.sub.evt")
# Create the description of output entities
dataDesc2  = DatasetDescription(id='muons_hdf5', name='muons', description='muon file', classType='datasetDescription')
wGBDesc1   = GenerationDescription(id='ctapipe_display_muons_0.6.1_muons_hdf5',activityDescription=actDesc1, entityDescription=dataDesc2, role="dl0.sub.evt")
valueDesc1 = ValueDescription(id='status', classType='valueDescription')
wGBDesc2   = GenerationDescription(id='ctapipe_display_muons_0.6.1_status', activityDescription=actDesc1, entityDescription=valueDesc1, role="quality")

# Put the instance in the database
session.add(actDesc1)
session.add(dataDesc1)
session.add(usedDesc1)
session.add(dataDesc2)
session.add(wGBDesc1)
session.add(valueDesc1)
session.add(wGBDesc2)
session.commit()

2019-08-01 11:44:47,690 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-01 11:44:47,691 INFO sqlalchemy.engine.base.Engine INSERT INTO "activityDescriptions" (id, name, version, description, type, subtype, doculink) VALUES (?, ?, ?, ?, ?, ?, ?)
2019-08-01 11:44:47,692 INFO sqlalchemy.engine.base.Engine ('ctapipe_display_muons_0.6.1', 'ctapipe_display_muons', '0.6.1', None, 'reconstruction', '', '')
2019-08-01 11:44:47,695 INFO sqlalchemy.engine.base.Engine INSERT INTO "entityDescriptions" (id, name, type, description, doculink, "classType") VALUES (?, ?, ?, ?, ?, ?)
2019-08-01 11:44:47,695 INFO sqlalchemy.engine.base.Engine (('proton_events', 'protons', None, 'proton file', None, 'datasetDescription'), ('muons_hdf5', 'muons', None, 'muon file', None, 'datasetDescription'), ('status', None, None, None, None, 'valueDescription'))
2019-08-01 11:44:47,697 INFO sqlalchemy.engine.base.Engine INSERT INTO "valueDescriptions" (id, "valueType", unit, ucd, utype, min, max, "default", 

## Run muon_reconstruction

In [4]:
from ctapipe.core import Provenance
from pprint import pprint
p = Provenance()  # note this is a singleton, so only ever one global provenence object
p.clear()

p.start_activity()
    
myTool = MuonDisplayerTool()
#myTool.run(['--input=gamma_20deg_180deg_run1000___cta-prod3-demo_desert-2150m-Paranal-demo2rad_cone10.simtel.gz'])
#print(myTool.get_current_config())

#myTool = MuonDisplayerTool()
myTool.run(['--input=proton_20deg_180deg_run22___cta-prod3-demo-2147m-LaPalma-baseline.simtel.gz'])

p.finish_activity()

INFO [MuonDisplayerTool] (tool/initialize): ctapipe version 0.5.2.post756+git93dae3f
INFO [MuonDisplayerTool] (tool/run): Starting: ctapipe-reconstruct-muons
INFO [MuonDisplayerTool] (tool/run): CONFIG: {'MuonDisplayerTool': {'config_file': '', 'display': False, 'events': 'proton_20deg_180deg_run22___cta-prod3-demo-2147m-LaPalma-baseline.simtel.gz', 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_format': '%(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s', 'log_level': 20, 'outfile': 'muons.hdf5'}}
detecting muons: 8it [00:02,  3.48it/s]
INFO [MuonDisplayerTool] (tool/run): Finished: ctapipe-reconstruct-muons
INFO [MuonDisplayerTool] (tool/run): Output: /Users/bourgeat/Documents/CTA/Provenance/ctasoft/improve-tool/ctapipe/prov_tests/muons.hdf5


In [5]:
p.finished_activity_names

['ctapipe-reconstruct-muons',
 '/Users/bourgeat/anaconda3/envs/cta-dev-improve/bin/python']

In [6]:
p.provenance[:-1]

[{'activity_name': 'ctapipe-reconstruct-muons',
  'activity_uuid': '4f0aac5b-8f39-453f-a8f6-9bac806c9fee',
  'start': {'time_utc': '2019-08-01T09:44:51.529'},
  'stop': {'time_utc': '2019-08-01T09:44:54.396'},
  'system': {'ctapipe_version': '0.5.2.post756+git93dae3f',
   'ctapipe_resources_version': '0.2.17',
   'pyhessio_version': 'not installed',
   'eventio_version': '0.20.3',
   'ctapipe_svc_path': None,
   'executable': '/Users/bourgeat/anaconda3/envs/cta-dev-improve/bin/python',
   'platform': {'architecture_bits': '64bit',
    'architecture_linkage': '',
    'machine': 'x86_64',
    'processor': 'i386',
    'node': 'pc33.home',
    'version': 'Darwin Kernel Version 17.7.0: Wed Apr 24 21:17:24 PDT 2019; root:xnu-4570.71.45~1/RELEASE_X86_64',
    'system': 'Darwin',
    'release': '17.7.0',
    'libcver': ('', ''),
    'num_cpus': 8,
    'boot_time': '2019-07-15T08:02:08.000'},
   'python': {'version_string': '3.7.3 (default, Mar 27 2019, 16:54:48) \n[Clang 4.0.1 (tags/RELEASE_40

## Provenance database update

In [7]:
import hashlib, uuid, os
BLOCKSIZE = 65536
hasher = hashlib.sha1()

def get_file_id(url):
    '''
    # Computation of the hash of the file to determine the id of it
    with open(cta_input['url'], 'rb') as afile:
        buf = afile.read(BLOCKSIZE)
        while len(buf) > 0:
            hasher.update(buf)
            buf = afile.read(BLOCKSIZE)
            return hasher.hexdigest()
    '''
    logical_name = url.split('/')[-1] 
    name = logical_name + str(os.path.getctime(url))
    file_uuid = str(uuid.uuid5(uuid.NAMESPACE_URL, name))
    if session.query(DatasetEntity).filter(DatasetEntity.id==file_uuid).count():
        print ("Existing Entity Key: ", logical_name)
        return file_uuid
    else:
        print ("New Entity Key: ", logical_name)
        return ""

    # universal unique id
    #return uuid.uuid4()

def set_file_id(url):
    logical_name = url.split('/')[-1] 
    name = logical_name + str(os.path.getctime(url))
    file_uuid = str(uuid.uuid5(uuid.NAMESPACE_URL, name))
    return file_uuid

def add_activity(session, cta_activity):
    if not (session.query(Activity).filter(Activity.id==cta_activity['activity_uuid']).count()): # for the tests
        current_activity = Activity(id=cta_activity['activity_uuid'])
        current_activity.name=cta_activity['activity_name']
        current_activity.startTime=cta_activity['start']['time_utc']
        current_activity.endTime=cta_activity['stop']['time_utc']
        current_activity.comment=''
        current_activity.activityDescription_id=cta_activity['activity_name']+'_'+cta_activity['system']['ctapipe_version']
        session.add(current_activity)
    
        # Association with the agent
        wAW = WasAssociatedWith()
        wAW.activity = cta_activity['activity_uuid']
        wAW.agent    = "CTAO"
        #wAW.role = ?
        session.add(wAW)

# CTAO Agent
CTA_org = "CTAO"
agent = Agent(id=CTA_org)
if session.query(Agent).filter(Agent.id==CTA_org).count():
    print ("Existing Agent Key: ", CTA_org)
    pass
else:
    agent.name ="CTA Observatory"
    agent.type = "Organization"
    session.add(agent)

# For each activity
for cta_activity in p.provenance[:-1]:
    add_activity(session, cta_activity)
    
    # For each input file
    for cta_input in cta_activity['input']:
        
        # Get the id of the file
        filename_uuid = get_file_id(cta_input['url'])
        print ("filename_uuid: ", filename_uuid)
            
        # If Entity does not exist in the database, add it - current_input_file.entityDescription_id= ???
        if filename_uuid == "":
            filename_uuid = set_file_id(cta_input['url'])
            current_input_file = DatasetEntity(id=filename_uuid, classType = 'dataset', \
                                    name = cta_input['url'].split('/')[-1], location = cta_input['url'])
            session.add(current_input_file)
            
        # Attribution to the agent - wAT.role = ?
        wAT = WasAttributedTo(entity = filename_uuid, agent = "CTAO")
        session.add(wAT)
            
        # Add the Used relationship
        used1 = Used(role = cta_input['role'], activity_id = cta_activity['activity_uuid'], entity_id = filename_uuid) # incremental id
        session.add(used1)
    
    # For each output file
    for cta_output in cta_activity['output']:
        
        # Computation of the hash of the file to determine the id of it
        filename_uuid = set_file_id(cta_output['url'])
        print ("filename_uuid: ", filename_uuid)
            
        # If Entity already exists in the database, raise an Exception or a error message - #current_output_file.entityDescription_id= ???
        if session.query(DatasetEntity).filter(DatasetEntity.id==filename_uuid).count():
            print ("ERROR")
        else:
            current_output_file = DatasetEntity(id=filename_uuid, classType = 'dataset', name = cta_output['url'].split('/')[-1],\
                                               location = cta_output['url'])
            session.add(current_output_file)
            
            # Attribution to the agent - wAT.role = ?
            wAT = WasAttributedTo(entity = filename_uuid, agent = "CTAO")
            session.add(wAT)
            
        # Add the wasgeneratedBy relationship - incremental id
        wGB1 = WasGeneratedBy(role = cta_output['role'], activity_id = cta_activity['activity_uuid'],\
                             entity_id = filename_uuid) 
        session.add(wGB1)
        
    # Add the status as an output ValueEntity
    current_output_value = ValueEntity(id=cta_activity['activity_uuid']+'_status')
    current_output_value.name = 'status'
    current_output_value.classType = 'value'
    current_output_value.valueXX = cta_activity['status']
    current_output_value.entityDescription_id = 'status'
    #current_output_value.location
    #current_output_value.entityDescription_id= ???
    session.add(current_output_value)
    
    # Add the wasgeneratedBy relationship - incremental id
    wGB2 = WasGeneratedBy(role = 'status', activity_id = cta_activity['activity_uuid'],\
                             entity_id = cta_activity['activity_uuid']+'_status') 
    session.add(wGB2)
        
session.commit()

2019-08-01 11:45:11,101 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-01 11:45:11,102 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT agents.id AS agents_id, agents.name AS agents_name, agents.type AS agents_type, agents.email AS agents_email, agents.affiliation AS agents_affiliation, agents.phone AS agents_phone, agents.address AS agents_address, agents.comment AS agents_comment, agents.url AS agents_url 
FROM agents 
WHERE agents.id = ?) AS anon_1
2019-08-01 11:45:11,103 INFO sqlalchemy.engine.base.Engine ('CTAO',)
2019-08-01 11:45:11,110 INFO sqlalchemy.engine.base.Engine INSERT INTO agents (id, name, type, email, affiliation, phone, address, comment, url) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-08-01 11:45:11,112 INFO sqlalchemy.engine.base.Engine ('CTAO', 'CTA Observatory', 'Organization', None, None, None, None, None, None)
2019-08-01 11:45:11,116 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT activities.id AS

## Store the items of the database in a file

In [8]:
# Put the results in a file for the Provenance RFC
with open("muons_provRFC.txt", "w") as prov:
    prov.write("Provenance working example - CTA ctapipe-display-muons\n")
    prov.write("datamodel version 1.2 / preparation for PR-version 2 January 2019. MS.\n")
    prov.write("=========================================================================\n")
    prov.write("Remarks\n")
    prov.write("- ActivityDescription id = activity_name + '_' + ctapipe version\n")
    prov.write("- Activity id = uuid returned from ctapipe\n")  
    prov.write("- Entity id = hash (file)\n")
    prov.write("- Link between Entity and EntityDescription not defined. Via role?\n")
    prov.write("- Used and WasGeneratedBy Ids = activity id + '_' + 'entity id or incremental?\n")
    prov.write("\n")
    prov.write("\n")
    prov.write("=========================================================================\n")
    prov.write("\n")

    # Opérations sur le fichier
    for classname in [ActivityDescription, EntityDescription, UsageDescription, GenerationDescription, \
                      DatasetDescription, ValueDescription, ParameterDescription,\
                      Activity, Entity, Used, WasGeneratedBy, \
                      DatasetEntity, ValueEntity, Parameter,\
                      Agent, WasAttributedTo, WasAssociatedWith]:
        for instance in session.query(classname).order_by(classname.id):
            prov.write("%s\n" %instance)
prov.close()

2019-08-01 11:45:17,707 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-01 11:45:17,709 INFO sqlalchemy.engine.base.Engine SELECT "activityDescriptions".id AS "activityDescriptions_id", "activityDescriptions".name AS "activityDescriptions_name", "activityDescriptions".version AS "activityDescriptions_version", "activityDescriptions".description AS "activityDescriptions_description", "activityDescriptions".type AS "activityDescriptions_type", "activityDescriptions".subtype AS "activityDescriptions_subtype", "activityDescriptions".doculink AS "activityDescriptions_doculink" 
FROM "activityDescriptions" ORDER BY "activityDescriptions".id
2019-08-01 11:45:17,710 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 11:45:17,712 INFO sqlalchemy.engine.base.Engine SELECT "entityDescriptions".id AS "entityDescriptions_id", "entityDescriptions".name AS "entityDescriptions_name", "entityDescriptions".type AS "entityDescriptions_type", "entityDescriptions".description AS "entityDescriptio

2019-08-01 11:45:17,760 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 11:45:17,767 INFO sqlalchemy.engine.base.Engine SELECT parameters.id AS parameters_id, parameters.value AS parameters_value, parameters.name AS parameters_name, parameters."parameterDescription_id" AS "parameters_parameterDescription_id" 
FROM parameters ORDER BY parameters.id
2019-08-01 11:45:17,769 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 11:45:17,771 INFO sqlalchemy.engine.base.Engine SELECT agents.id AS agents_id, agents.name AS agents_name, agents.type AS agents_type, agents.email AS agents_email, agents.affiliation AS agents_affiliation, agents.phone AS agents_phone, agents.address AS agents_address, agents.comment AS agents_comment, agents.url AS agents_url 
FROM agents ORDER BY agents.id
2019-08-01 11:45:17,773 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 11:45:17,776 INFO sqlalchemy.engine.base.Engine SELECT "wasAttributedTo".id AS "wasAttributedTo_id", "wasAttributedTo".role AS "wasAttributed

In [9]:
# Display the file contents
with open("muons_provRFC.txt", "r") as prov:
    for line in prov:
        print (line[:-1])
prov.close()

Provenance working example - CTA ctapipe-display-muons
datamodel version 1.2 / preparation for PR-version 2 January 2019. MS.
Remarks
- ActivityDescription id = activity_name + '_' + ctapipe version
- Activity id = uuid returned from ctapipe
- Entity id = hash (file)
- Link between Entity and EntityDescription not defined. Via role?
- Used and WasGeneratedBy Ids = activity id + '_' + 'entity id or incremental?



ActivityDescription.id=ctapipe_display_muons_0.6.1
ActivityDescription.name=ctapipe_display_muons
ActivityDescription.version=0.6.1
ActivityDescription.description=None
ActivityDescription.type=reconstruction
ActivityDescription.subtype=
ActivityDescription.doculink=

DatasetDescription.id=muons_hdf5
DatasetDescription.name=muons
DatasetDescription.type=None
DatasetDescription.description=muon file
DatasetDescription.doculink=None
DatasetDescription.classType=datasetDescription

DatasetDescription.id=proton_events
DatasetDescription.name=protons
DatasetDescription.type=None
Da

## Visualize the provenance

In [10]:
import  prov.model
import  prov.dot

provDoc = prov.model.ProvDocument()
provDoc.add_namespace('voprov', 'http://wiki.ivoa.net/twiki/bin/view/IVOA/ProvenanceDataModel/ns/')
provDoc.add_namespace('prov', 'http://www.w3.org/ns/prov/')
provDoc.add_namespace('cta', 'http://voparis-cta-confluence.obspm.fr/provenance/')
provFile = "muons_provRFC.svg"

for classname in [Activity]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.activity('cta:' + instance.id, startTime=instance.startTime, endTime=instance.endTime)
for classname in [Entity, DatasetEntity]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.entity('cta:' + str(instance.id), {'voprov:name':instance.name})
for classname in [ValueEntity]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.entity('cta:' + str(instance.id), {'voprov:name':instance.name, 'voprov:value':instance.valueXX})
for classname in [Used]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.used('cta:'+instance.activity_id, 'cta:'+str(instance.entity_id))
for classname in [WasGeneratedBy]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.wasGeneratedBy('cta:'+str(instance.entity_id), 'cta:'+instance.activity_id)
for classname in [Agent]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.agent('cta:'+instance.id)
for classname in [WasAssociatedWith]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.wasAssociatedWith('cta:'+instance.activity, 'cta:'+instance.agent)
for classname in [WasAttributedTo]:
    for instance in session.query(classname).order_by(classname.id):
        provDoc.wasAttributedTo('cta:'+instance.entity, 'cta:'+instance.agent)

dot = prov.dot.prov_to_dot(provDoc, use_labels=True)
dot.write_svg(provFile)



2019-08-01 11:45:33,116 INFO sqlalchemy.engine.base.Engine SELECT activities.id AS activities_id, activities.name AS activities_name, activities."startTime" AS "activities_startTime", activities."endTime" AS "activities_endTime", activities.comment AS activities_comment, activities."activityDescription_id" AS "activities_activityDescription_id" 
FROM activities ORDER BY activities.id
2019-08-01 11:45:33,117 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 11:45:33,119 INFO sqlalchemy.engine.base.Engine SELECT entities.id AS entities_id, entities.name AS entities_name, entities.location AS entities_location, entities."generatedAtTime" AS "entities_generatedAtTime", entities."invalidatedAtTime" AS "entities_invalidatedAtTime", entities.comment AS entities_comment, entities."entityDescription_id" AS "entities_entityDescription_id", entities."classType" AS "entities_classType" 
FROM entities ORDER BY entities.id
2019-08-01 11:45:33,119 INFO sqlalchemy.engine.base.Engine ()
2019-08-01 11:45

## Display the provenance 
<img src="muons_provRFC.svg?modified=1245678901">

In [17]:
session.close()

2019-07-31 14:57:09,428 INFO sqlalchemy.engine.base.Engine ROLLBACK
